<a href="https://colab.research.google.com/github/gmmanzo/Seizure-Detection/blob/main/Seizure_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a seizure detection model using Time Series forecasting with the folllowing dataset https://physionet.org/content/chbmit/1.0.0/chb01/#files-panel